### constant value

In [1]:
import glob
import numpy as np
import math
import os

In [2]:
def eulerRad2mtx(data):
    posX = data[0]
    posY = data[1]
    posZ = data[2]
    pitch = data[3]
    roll = data[4]
    yaw = data[5]
    
    Rx = np.array([[1, 0, 0, 0],
                   [0, math.cos(pitch), -1 * math.sin(pitch), 0],
                   [0, math.sin(pitch), math.cos(pitch), 0],
                   [0, 0, 0, 1]])
    
    Ry = np.array([[math.cos(roll), 0, math.sin(roll), 0],
                   [0, 1, 0, 0],
                   [-1 * math.sin(roll), 0, math.cos(roll), 0],
                   [0, 0, 0, 1]])
    
    Rz = np.array([[math.cos(yaw), -1 * math.sin(yaw), 0, 0],
                   [math.sin(yaw), math.cos(yaw), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    
    mtxR = np.dot(Rz, np.dot(Ry, Rx))
    
    mtxR[0][3] = posX
    mtxR[1][3] = posY
    mtxR[2][3] = posZ
    
    return mtxR

def mtx2eulerDeg(mtx) :
    dLocSqrt = (mtx[0][0] * mtx[0][0] +  mtx[1][0] * mtx[1][0])
    
    if(dLocSqrt<0.0):
        dLocSqrt = 0.0
    else:
        dLocSqrt= math.sqrt(dLocSqrt)
    
    dLocRy = math.atan2(-mtx[2][0], dLocSqrt)
    dLocCy = math.cos(dLocRy)
 
    if (dLocCy < 0.001745328365898):
        Rx = 0.0
        Ry = dLocRy
        Rz = math.atan2(-mtx[0][1], mtx[1][1])
    else:
        Rx = math.atan2(mtx[2][1]/dLocCy, mtx[2][2]/dLocCy)
        Ry = dLocRy
        Rz = math.atan2(mtx[1][0]/dLocCy, mtx[0][0]/dLocCy)
        
    X = mtx[0][3]
    Y = mtx[1][3]
    Z = mtx[2][3]
    Rx = Rx * 180 / 3.141592653589793238462643383279
    Ry = Ry * 180 / 3.141592653589793238462643383279
    Rz = Rz * 180 / 3.141592653589793238462643383279
 
    return np.array([X, Y, Z, Rx, Ry, Rz])

In [3]:
def mulmulmtx(robot2base, ots2base, ots2marker, marker2bone, bone2imp):
    temp0 = np.dot(robot2base, np.linalg.inv(ots2base))
    temp1 = np.dot(temp0, ots2marker)
    temp2 = np.dot(temp1, marker2bone)
    temp3 = np.dot(temp2, bone2imp)
    
    result = temp3
    
    return result

In [4]:
robot2robotBaseMrk = []

femurMrk2femurBone = []
tibiaMrk2tibiaBone = []

femurBone2femurImp = []
tibiaBone2tibiaImp = []

In [5]:
mtx_lists = glob.glob(os.path.join("data", "*.txt"))

In [6]:
for file_name in mtx_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
    
    if "Robot" in file_name:
        for line in line_data:
            robot2robotBaseMrk.append(line.split('\n')[0].split(','))
    elif "Femur." in file_name:
        for line in line_data:
            femurMrk2femurBone.append(line.split('\n')[0].split(','))
    elif "Tibia." in file_name:
        for line in line_data:
            tibiaMrk2tibiaBone.append(line.split('\n')[0].split(','))
    elif "Femur_Imp" in file_name:
        for line in line_data:
            femurBone2femurImp.append(line.split('\n')[0].split(','))
    elif "Tibia_Imp" in file_name:
        for line in line_data:
            tibiaBone2tibiaImp.append(line.split('\n')[0].split(','))

data\Transform_BaseMarker2Robot.txt
data\Transform_Femur.txt
data\Transform_Femur_Imp.txt
data\Transform_Tibia.txt
data\Transform_Tibia_Imp.txt


In [7]:
robot2robotBaseMrk = np.array(robot2robotBaseMrk, dtype='float')

femurMrk2femurBone = np.array(femurMrk2femurBone, dtype='float')
tibiaMrk2tibiaBone = np.array(tibiaMrk2tibiaBone, dtype='float')

femurBone2femurImp = np.array(femurBone2femurImp, dtype='float')
tibiaBone2tibiaImp = np.array(tibiaBone2tibiaImp, dtype='float')

### raw mrk data

###### < 0.15

In [8]:
pass_v0_f_OTS2RobotMrk = []
pass_v0_OTS2FemurMrk = []

In [9]:
euler_pass_lists = glob.glob(os.path.join("data/00_pass/00_f", "*.txt"))

In [10]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            pass_v0_f_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "femur" in file_name:
        for line in line_data:
            pass_v0_OTS2FemurMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/00_pass/00_f\2020-09-01-15-58-01_base_Euler_points.txt
data/00_pass/00_f\2020-09-01-15-58-01_femur_Euler_points.txt


In [11]:
pass_v0_t_OTS2RobotMrk = []
pass_v0_OTS2TibiaMrk = []

In [12]:
euler_pass_lists = glob.glob(os.path.join("data/00_pass/01_t", "*.txt"))

In [13]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            pass_v0_t_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "tibia" in file_name:
        for line in line_data:
            pass_v0_OTS2TibiaMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/00_pass/01_t\2020-09-01-15-58-01_base_Euler_points.txt
data/00_pass/01_t\2020-09-01-15-58-01_tibia_Euler_points.txt


In [14]:
femur_v0_imp_crd_euler = []
tibia_v0_imp_crd_euler = []

for i in range(len(pass_v0_f_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, pass_v0_f_OTS2RobotMrk[i], pass_v0_OTS2FemurMrk[i], femurMrk2femurBone, femurBone2femurImp)
    femur_v0_imp_crd_euler.append(mtx2eulerDeg(tmp))
    
for i in range(len(pass_v0_t_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, pass_v0_t_OTS2RobotMrk[i], pass_v0_OTS2TibiaMrk[i], tibiaMrk2tibiaBone, tibiaBone2tibiaImp)
    tibia_v0_imp_crd_euler.append(mtx2eulerDeg(tmp))

###### 0.15 < x < 0.30

In [15]:
fail_v1_f_OTS2RobotMrk = []
fail_v1_OTS2FemurMrk = []

In [16]:
euler_pass_lists = glob.glob(os.path.join("data/02_femur_fail/02", "*.txt"))

In [17]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            fail_v1_f_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "femur" in file_name:
        for line in line_data:
            fail_v1_OTS2FemurMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/02_femur_fail/02\2020-09-01-16-16-49_base_Euler_points.txt
data/02_femur_fail/02\2020-09-01-16-16-49_femur_Euler_points.txt


In [18]:
fail_v1_t_OTS2RobotMrk = []
fail_v1_OTS2TibiaMrk = []

In [19]:
euler_pass_lists = glob.glob(os.path.join("data/03_tibia_fail/02", "*.txt"))

In [20]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            fail_v1_t_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "tibia" in file_name:
        for line in line_data:
            fail_v1_OTS2TibiaMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/03_tibia_fail/02\2020-09-01-16-01-13_base_Euler_points.txt
data/03_tibia_fail/02\2020-09-01-16-01-13_tibia_Euler_points.txt


In [21]:
femur_v1_imp_crd_euler = []
tibia_v1_imp_crd_euler = []

for i in range(len(fail_v1_f_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, fail_v1_f_OTS2RobotMrk[i], fail_v1_OTS2FemurMrk[i], femurMrk2femurBone, femurBone2femurImp)
    femur_v1_imp_crd_euler.append(mtx2eulerDeg(tmp))
    
for i in range(len(fail_v1_t_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, fail_v1_t_OTS2RobotMrk[i], fail_v1_OTS2TibiaMrk[i], tibiaMrk2tibiaBone, tibiaBone2tibiaImp)
    tibia_v1_imp_crd_euler.append(mtx2eulerDeg(tmp))

###### > 0.30

In [22]:
fail_v2_f_OTS2RobotMrk = []
fail_v2_OTS2FemurMrk = []

In [23]:
euler_pass_lists = glob.glob(os.path.join("data/02_femur_fail/03", "*.txt"))

In [24]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            fail_v2_f_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "femur" in file_name:
        for line in line_data:
            fail_v2_OTS2FemurMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/02_femur_fail/03\2020-09-01-16-16-59_base_Euler_points.txt
data/02_femur_fail/03\2020-09-01-16-16-59_femur_Euler_points.txt


In [25]:
fail_v2_t_OTS2RobotMrk = []
fail_v2_OTS2TibiaMrk = []

In [26]:
euler_pass_lists = glob.glob(os.path.join("data/03_tibia_fail/03", "*.txt"))

In [27]:
for file_name in euler_pass_lists:
    print(file_name)
    
    line_data = []
    
    with open(file_name, "r", encoding='UTF8') as f:
        line_data = f.readlines()
        
    if "base" in file_name:
        for line in line_data:
            fail_v2_t_OTS2RobotMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))
    elif "tibia" in file_name:
        for line in line_data:
            fail_v2_OTS2TibiaMrk.append(eulerRad2mtx(np.array(line.split('\n')[0].split(','), dtype='float')))

data/03_tibia_fail/03\2020-09-01-16-02-53_base_Euler_points.txt
data/03_tibia_fail/03\2020-09-01-16-02-53_tibia_Euler_points.txt


In [28]:
femur_v2_imp_crd_euler = []
tibia_v2_imp_crd_euler = []

for i in range(len(fail_v2_f_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, fail_v2_f_OTS2RobotMrk[i], fail_v2_OTS2FemurMrk[i], femurMrk2femurBone, femurBone2femurImp)
    femur_v2_imp_crd_euler.append(mtx2eulerDeg(tmp))
    
for i in range(len(fail_v2_t_OTS2RobotMrk)):
    tmp = mulmulmtx(robot2robotBaseMrk, fail_v2_t_OTS2RobotMrk[i], fail_v2_OTS2TibiaMrk[i], tibiaMrk2tibiaBone, tibiaBone2tibiaImp)
    tibia_v2_imp_crd_euler.append(mtx2eulerDeg(tmp))

#### 정리!

In [29]:
femur_v0_imp_crd_euler
tibia_v0_imp_crd_euler
femur_v1_imp_crd_euler
tibia_v1_imp_crd_euler
femur_v2_imp_crd_euler
tibia_v2_imp_crd_euler

[array([ 616.43328531, -209.87226241,  195.14384242, -128.24631732,
           6.75140138, -179.94028207]),
 array([ 615.9461995 , -210.1188125 ,  194.87817908, -128.24526698,
           6.72205939, -179.89832643]),
 array([ 615.6032212 , -210.388566  ,  195.19810801, -128.2449674 ,
           6.73421705, -179.9812598 ]),
 array([ 616.08488363, -209.93016971,  194.72104814, -128.26106575,
           6.65851615, -179.91930351]),
 array([ 615.81635736, -210.02941179,  194.83731145, -128.23942213,
           6.66066215, -179.93214069]),
 array([ 616.32332746, -209.80659413,  194.91700575, -128.23955987,
           6.68841672, -179.92343287]),
 array([ 615.90516919, -209.96998954,  194.76586498, -128.23398454,
           6.66433164, -179.96157893]),
 array([ 616.16422646, -209.80823276,  194.80265841, -128.24269118,
           6.67061839, -179.94760502]),
 array([ 615.69402154, -210.09264236,  194.72226873, -128.24413347,
           6.70164688, -179.94270706]),
 array([ 615.32241308, -210.

In [30]:
def print_euler_np_array(np_array):
    for arr in np_array:
        str_data = ""
        for elm in arr:
            str_data = str_data + str(elm) + ', '
        print(str_data[:-2])

In [31]:
print_euler_np_array(femur_v0_imp_crd_euler)
print('\n')
print_euler_np_array(tibia_v0_imp_crd_euler)
print('\n')
print_euler_np_array(femur_v1_imp_crd_euler)
print('\n')
print_euler_np_array(tibia_v1_imp_crd_euler)
print('\n')
print_euler_np_array(femur_v2_imp_crd_euler)
print('\n')
print_euler_np_array(tibia_v2_imp_crd_euler)

601.4091836057117, -148.24232288328898, 263.76489598890606, -154.1496727791568, -10.660306593362366, -0.9770608300292912
600.7619257661697, -148.69726420964759, 263.69836952684125, -154.1560019586136, -10.654742295915954, -0.9989885085553565
601.7213447208869, -148.24370530770142, 263.8815520656629, -154.1664325725781, -10.718562833931752, -0.8964225744020816
600.6177977894578, -148.568455123077, 263.56499354220693, -154.15425511601097, -10.71543784985411, -1.0549710921711026
600.9642156312523, -148.54781166900233, 263.6794455930895, -154.16578426822065, -10.719318387450217, -0.9924949621381743
601.0324365416609, -148.67451586677623, 263.857918678245, -154.16147560077474, -10.69847924690413, -0.9919365156654749
601.1886831743001, -148.5362437316185, 263.79693983879434, -154.15942777394565, -10.714133669591037, -0.9763072491320396
600.6562867224204, -149.15738277737125, 264.05203420344856, -154.1857040301312, -10.738219900678013, -0.9724337606570631
601.018733962152, -148.72804872424004

### dist.

In [32]:
femur_v0_imp_crd_euler_dist = []
tibia_v0_imp_crd_euler_dist = []
femur_v1_imp_crd_euler_dist = []
tibia_v1_imp_crd_euler_dist = []
femur_v2_imp_crd_euler_dist = []
tibia_v2_imp_crd_euler_dist = []

In [33]:
def euler_2_dist(euler):
    x = euler[0]
    y = euler[1]
    z = euler[2]
    
    return math.sqrt(pow(x, 2) + pow(y, 2) + pow(z, 2))

def euler_list_2_mean_diff_and_dist(euler_list):
    mean_diff_and_dist = []
    
    tmp_sum = np.array([0, 0, 0, 0, 0, 0])

    for el in euler_list:
        tmp_sum = tmp_sum + el
        
    tmp_avr = tmp_sum / len(euler_list)
    mean_diff_and_dist.append(tmp_avr.tolist())
    
    for el in euler_list:
        mean_diff_and_dist.append((tmp_avr - el).tolist())
        
    for i in range(len(mean_diff_and_dist)):
        if i == 0:
            mean_diff_and_dist[i].append(0)
        else:
            mean_diff_and_dist[i].append(euler_2_dist(mean_diff_and_dist[i]))
            
    return mean_diff_and_dist

In [34]:
femur_v0_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(femur_v0_imp_crd_euler)
tibia_v0_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(tibia_v0_imp_crd_euler)
femur_v1_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(femur_v1_imp_crd_euler)
tibia_v1_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(tibia_v1_imp_crd_euler)
femur_v2_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(femur_v2_imp_crd_euler)
tibia_v2_imp_crd_euler_dist = euler_list_2_mean_diff_and_dist(tibia_v2_imp_crd_euler)

### export to xlsx

In [35]:
case_name = 'case1'

In [36]:
head_data = ['x < 0.15', '0.15 < x < 0.3', 'x > 0.3']
head_bone_data = ['femur', 'tibia']

data_list = [femur_v0_imp_crd_euler,
             tibia_v0_imp_crd_euler,
             femur_v1_imp_crd_euler,
             tibia_v1_imp_crd_euler,
             femur_v2_imp_crd_euler,
             tibia_v2_imp_crd_euler]

data_list_dist = [femur_v0_imp_crd_euler_dist,
                  tibia_v0_imp_crd_euler_dist,
                  femur_v1_imp_crd_euler_dist,
                  tibia_v1_imp_crd_euler_dist,
                  femur_v2_imp_crd_euler_dist,
                  tibia_v2_imp_crd_euler_dist]

In [37]:
import pandas as pd
from pandas import DataFrame

In [38]:
df = pd.DataFrame()

In [39]:
count = 0

for i in range(len(head_data)):
    for j in range(len(head_bone_data)):
        df_head = pd.DataFrame([[head_data[i], head_bone_data[j], ' ', ' ', ' ', ' ']], 
                               columns = ['Tx', 'Ty', 'Tz', 'Rx', 'Ry', 'Rz'])

        df_data = pd.DataFrame(data_list[count], columns = ['Tx', 'Ty', 'Tz', 'Rx', 'Ry', 'Rz'])

        df_tmp = pd.concat([df_head, df_data])
        
        df = pd.concat([df, df_tmp])
        
        count = count + 1

In [40]:
df

,Tx,Ty,Tz,Rx,Ry,Rz
0,x < 0.15,femur,,,,
0,601.409,-148.242,263.765,-154.15,-10.6603,-0.977061
1,600.762,-148.697,263.698,-154.156,-10.6547,-0.998989
2,601.721,-148.244,263.882,-154.166,-10.7186,-0.896423
3,600.618,-148.568,263.565,-154.154,-10.7154,-1.05497
...,...,...,...,...,...,...
15,615.846,-209.934,194.84,-128.259,6.81115,179.994
16,615.724,-210.087,194.909,-128.261,6.77967,179.917
17,616.053,-209.888,194.816,-128.244,6.77331,-179.975
18,615.388,-210.235,194.678,-128.24,6.77869,-179.92


In [41]:
df.to_excel('00_' + case_name + '.xlsx')

In [42]:
dfd = pd.DataFrame()

In [43]:
count = 0

for i in range(len(head_data)):
    for j in range(len(head_bone_data)):
        df_head = pd.DataFrame([[head_data[i], head_bone_data[j], ' ', ' ', ' ', ' ', ' ']], 
                               columns = ['diff. Tx', 'diff. Ty', 'diff. Tz', 'diff. Rx', 'diff. Ry', 'diff. Rz', 'dist.'])

        df_data = pd.DataFrame(data_list_dist[count], 
                               columns = ['diff. Tx', 'diff. Ty', 'diff. Tz', 'diff. Rx', 'diff. Ry', 'diff. Rz', 'dist.'])

        df_tmp = pd.concat([df_head, df_data])
        
        dfd = pd.concat([dfd, df_tmp])
        
        count = count + 1

In [44]:
dfd

,diff. Tx,diff. Ty,diff. Tz,diff. Rx,diff. Ry,diff. Rz,dist.
0,x < 0.15,femur,,,,,
0,601.129,-148.541,263.768,-154.161,-10.6841,-0.970433,0
1,-0.280612,-0.29894,0.00344869,-0.0115243,-0.0237609,0.00662809,0.410025
2,0.366646,0.156001,0.0699752,-0.00519511,-0.0293252,0.0285558,0.404552
3,-0.592773,-0.297557,-0.113207,0.00523551,0.0344953,-0.0740102,0.672857
...,...,...,...,...,...,...,...
16,-0.104683,-0.181505,0.0539642,0.0121421,-0.0655534,-287.959,0.216367
17,0.0173092,-0.0285608,-0.0150259,0.0143938,-0.0340693,-287.881,0.0366211
18,-0.311067,-0.227331,0.0777364,-0.00261222,-0.0277159,72.0106,0.393046
19,0.353593,0.119553,0.216157,-0.00589251,-0.0330956,71.9555,0.431329


In [45]:
dfd.to_excel('00_' + case_name + '_dist.xlsx')

In [46]:
## XlsxWriter 엔진으로 Pandas writer 객체 만들기
writer = pd.ExcelWriter('00_' + case_name + '_with_chart.xlsx', engine='xlsxwriter')
 
    
trial_num = 6

trg_chart_point = ['A', 'B', 'C']
trg_start_num = 3
trg_end_num = 22

chart_start_point = ['H', 'P', 'X']

add_row_num = 21
    
## DataFrame을 xlsx에 쓰기
df.to_excel(writer, sheet_name='Sheet1', index=False)

## Pandas writer 객체에서 xlsxwriter 객체 가져오기
workbook = writer.book
worksheet = writer.sheets['Sheet1']
 
for i in range(trial_num):
    for j in range(len(trg_chart_point)):
        ## Chart 객체 만들기
        chart = workbook.add_chart({'type':'line'})

        ## 데이터 고르기
        chart.add_series({'values':'=Sheet1!' + trg_chart_point[j] + str(trg_start_num) + ':' + trg_chart_point[j] + str(trg_end_num)})

        ## Chart 넣기
        worksheet.insert_chart(chart_start_point[j] + str(trg_start_num), chart)
        
    trg_start_num = trg_start_num + add_row_num
    trg_end_num = trg_end_num + add_row_num
    
    
    
trg_start_num_ = 4
trg_end_num_ = 23

add_row_num_ = 22

## DataFrame을 xlsx에 쓰기
dfd.to_excel(writer, sheet_name='Sheet2', index=False)

## Pandas writer 객체에서 xlsxwriter 객체 가져오기
workbook = writer.book
worksheet = writer.sheets['Sheet2']
 
for i in range(trial_num):
    ## Chart 객체 만들기
    chart = workbook.add_chart({'type':'line'})

    ## 데이터 고르기
    chart.add_series({'values':'=Sheet2!' + 'G' + str(trg_start_num_) + ':' + 'G' + str(trg_end_num_)})

    ## Chart 넣기
    worksheet.insert_chart('J' + str(trg_start_num_), chart)
        
    trg_start_num_ = trg_start_num_ + add_row_num_
    trg_end_num_ = trg_end_num_ + add_row_num_

    
    
## Pandas writer 객체 닫기
writer.close()